<a href="https://colab.research.google.com/github/Ekaagra08/CNN-LSTM_video_classification/blob/main/cnn_lstm_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q imageio
!pip install -q opencv-python
!pip install -q git+https://github.com/tensorflow/docs

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import *
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import os
import math

# TensorFlow and TF-Hub modules.
from absl import logging
import tensorflow_hub as hub
from tensorflow_docs.vis import embed

logging.set_verbosity(logging.ERROR)

# Some modules to help with reading the UCF101 dataset.
import random
import re
import tempfile
import ssl

# Some modules to display an animation using imageio.
import imageio
from IPython import display

from urllib import request  # requires python3

# loading UCF101 dataset

In [3]:
!wget --no-check-certificate https://www.crcv.ucf.edu/data/UCF101/UCF101.rar
!wget -q --no-check-certificate https://www.crcv.ucf.edu/data/UCF101/UCF101TrainTestSplits-RecognitionTask.zip

--2021-11-26 16:16:09--  https://www.crcv.ucf.edu/data/UCF101/UCF101.rar
Resolving www.crcv.ucf.edu (www.crcv.ucf.edu)... 132.170.214.127
Connecting to www.crcv.ucf.edu (www.crcv.ucf.edu)|132.170.214.127|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 6932971618 (6.5G) [application/rar]
Saving to: ‘UCF101.rar’

UCF101.rar          100%[===================>]   6.46G  36.1MB/s    in 2m 15s  

2021-11-26 16:18:24 (48.9 MB/s) - ‘UCF101.rar’ saved [6932971618/6932971618]



In [6]:
%%capture
!unrar x UCF101.rar data/
!unzip -qq UCF101TrainTestSplits-RecognitionTask.zip

#dataset prepration and pre-preprocessingṁ

In [7]:
image_size= 64 # ie. 64*64

#number of frames to passs at a time in a sequence
sequence_len = 30

#data directoy
dataset_dir="/content/data/UCF-101"

#selected classes
calss_names_path = "/content/ucfTrainTestlist/classInd.txt"
class_list = []
with open(calss_names_path,"r") as f:
  for line in f:
    class_list.append(line.split()[1])

f.close()

print(class_list)

#subset list that we wil work on
selected_class_list = ["Swing","TaiChi","HorseRace"]

['ApplyEyeMakeup', 'ApplyLipstick', 'Archery', 'BabyCrawling', 'BalanceBeam', 'BandMarching', 'BaseballPitch', 'Basketball', 'BasketballDunk', 'BenchPress', 'Biking', 'Billiards', 'BlowDryHair', 'BlowingCandles', 'BodyWeightSquats', 'Bowling', 'BoxingPunchingBag', 'BoxingSpeedBag', 'BreastStroke', 'BrushingTeeth', 'CleanAndJerk', 'CliffDiving', 'CricketBowling', 'CricketShot', 'CuttingInKitchen', 'Diving', 'Drumming', 'Fencing', 'FieldHockeyPenalty', 'FloorGymnastics', 'FrisbeeCatch', 'FrontCrawl', 'GolfSwing', 'Haircut', 'Hammering', 'HammerThrow', 'HandstandPushups', 'HandstandWalking', 'HeadMassage', 'HighJump', 'HorseRace', 'HorseRiding', 'HulaHoop', 'IceDancing', 'JavelinThrow', 'JugglingBalls', 'JumpingJack', 'JumpRope', 'Kayaking', 'Knitting', 'LongJump', 'Lunges', 'MilitaryParade', 'Mixing', 'MoppingFloor', 'Nunchucks', 'ParallelBars', 'PizzaTossing', 'PlayingCello', 'PlayingDaf', 'PlayingDhol', 'PlayingFlute', 'PlayingGuitar', 'PlayingPiano', 'PlayingSitar', 'PlayingTabla', 'P

In [8]:
# Helper functions

#crops center portioin of frame
def crop_center_square(frame):
  y, x = frame.shape[0:2]
  min_dim = min(y, x)
  start_x = (x // 2) - (min_dim // 2)
  start_y = (y // 2) - (min_dim // 2)
  return frame[start_y:start_y+min_dim,start_x:start_x+min_dim]

######################################################################################  

# returns normalized numpy array of each frame(image) in video(sequence of images) 
def extract_frames(path, max_frames=0, resize=(224, 224)):
  cap = cv2.VideoCapture(path)
  frames_list = []
  #frameRate = cap.get(5) #frame rate 
  video_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  skip_frame_window = max(int(video_frame_count / sequence_len),1)
  
  try:
    for frame_counter in range(sequence_len):
      cap.set(cv2.CAP_PROP_POS_FRAMES, frame_counter*skip_frame_window) #current frame number
      
      ret, frame = cap.read()
      
      if not ret:
        break

      #if (frameId % math.floor(frameRate) == 0):
      croped_frame = crop_center_square(frame) #crop frame
      resized_frame = cv2.resize(croped_frame, resize) #resize frame
      frame = frame[:, :, [2, 1, 0]]
      frames_list.append(resized_frame)

      if len(frames_list ) == max_frames:
        break
  finally:
    cap.release()
  print("[INFO] {} Frames extracted".format(len(frames_list)))
  # return normalized frames list 
  return np.array(frames_list ) / 255.0

######################################################################################  


def to_gif(images):
  converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
  imageio.mimsave('./animation.gif', converted_images, fps=25)
  return embed.embed_file('./animation.gif')

######################################################################################  



In [9]:
to_gif(extract_frames("/content/data/UCF-101/BaseballPitch/v_BaseballPitch_g01_c03.avi")) 

[INFO] 30 Frames extracted


KeyboardInterrupt: ignored

#Create Dataset

In [ ]:
def create_dataset():
  features=[] #list of multiple lists of frames from one video class (X)
  lables=[] #list of class lables (y)
  video_file_paths=[]

  for class_index, class_name in enumerate(selected_class_list):
    print("[INFO] Exgtracting data of class: {}".format(class_name))

    #list of all video files in one class directory
    files_list = os.listdir(os.path.join(dataset_dir, class_name))

    #iterating through all files in class
    for file_name in files_list:
      #get complete video path
      video_file_path = os.path.join(dataset_dir, class_name, file_name)

      #extracting frames of the video
      frames = extract_frames(video_file_path)
      
      #select if no. of frames match sequence length else dont select
     # if len(frames) == sequence_len:
      features.append(frames)
      lables.append(class_index)
      video_file_paths.append(video_file_path)
  return features, lables, video_file_paths         

In [ ]:
features, labels, video_file_paths = create_dataset()

data = []
labels = []
#### loop over the image paths
for imagePath in imagePaths:
    # extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2]
    # if the label of the current image is not part of of the labels
    # are interested in, then ignore the image
    if label not in LABELS:
        continue
    # load the image, convert it to RGB channel ordering, and resize
    # it to be a fixed 224x224 pixels, ignoring aspect ratio
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    # update the data and labels lists, respectively
    data.append(image)
    labels.append(label)

In [ ]:
len(features[1])

In [ ]:
# convert the data and labels to NumPy arrays
features = np.array(features)
labels = np.array(labels)
# perform one-hot encoding on the labels
one_hot_encoded_labels = to_categorical(labels)
#lb = LabelBinarizer()
#labels = lb.fit_transform(labels)
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(features, labels,
	test_size=0.25,shuffle=True, stratify=labels, random_state=42)

In [ ]:
def create_cnn-lstm_model():
  model=Sequential()

  model.add(ConvLSTM2D(filters=4, kernel_size=(3,3), activation='tanh', data_format='channels_last', 
                       recurrent_dropout=0.2, return_sequences=True, input_shape = (sequence_len,image_size,image_size,3)))
  model.add(MaxPooling3D(pool_size=(1,2,2), padding='same', data_format='channels_last'))
  model.add(TimeDistributed(Dropout(.02)))
  

  model.add(ConvLSTM2D(filters=8, kernel_size=(3,3), activation='tanh', data_format='channels_last', 
                       recurrent_dropout=0.2, return_sequences=True))
  model.add(MaxPooling3D(pool_size=(1,2,2), padding='same', data_format='channels_last'))
  model.add(TimeDistributed(Dropout(.02)))
  
  
  model.add(ConvLSTM2D(filters=14, kernel_size=(3,3), activation='tanh', data_format='channels_last', 
                       recurrent_dropout=0.2, return_sequences=True))
  model.add(MaxPooling3D(pool_size=(1,2,2), padding='same', data_format='channels_last'))
  model.add(TimeDistributed(Dropout(.02)))
  
  
  model.add(ConvLSTM2D(filters=16, kernel_size=(3,3), activation='tanh', data_format='channels_last', 
                       recurrent_dropout=0.2, return_sequences=True))
  model.add(MaxPooling3D(pool_size=(1,2,2), padding='same', data_format='channels_last'))
  #model.add(TimeDistributed(Dropout(.02)))

  model.add(Flatten())

  model.add(Dense(len(selected_class_list), activation='softmax'))

  model.summary

  return model

In [ ]:
convlstm_model= create_cnn-lstm_model()
print("model created")

# initialize the training data augmentation object
trainAug = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")
# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()
# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

# load the ResNet-50 network, ensuring the head FC layer sets are left
# off
baseModel = ResNet50(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(lb.classes_), activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the training process
for layer in baseModel.layers:
	layer.trainable = False

# compile our model (this needs to be done after our setting our
# layers to being non-trainable)
print("[INFO] compiling model...")
opt = SGD(lr=1e-4, momentum=0.9, decay=1e-4 / args["epochs"])
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
print("[INFO] training head...")
H = model.fit(
	x=trainAug.flow(trainX, trainY, batch_size=32),
	steps_per_epoch=len(trainX) // 32,
	validation_data=valAug.flow(testX, testY),
	validation_steps=len(testX) // 32,
	epochs=args["epochs"])

In [ ]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(x=testX.astype("float32"), batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=lb.classes_))
# plot the training loss and accuracy
N = args["epochs"]
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(args["plot"])

[INFO] evaluating network...


NameError: ignored

In [ ]:
# serialize the model to disk
print("[INFO] serializing network...")
model.save(args["model"], save_format="h5")
# serialize the label binarizer to disk
f = open(args["label_bin"], "wb")
f.write(pickle.dumps(lb))
f.close()

[INFO] serializing network...


NameError: ignored